# <h1 align="center" style="font-size:200%; color:blue">Microsoft’s entrance to Movie business</h1><br>
<h1 align="center" style="font-size:200%; color:blue">Data loading and cleaning and exploration</h1> <br>

<p style="color:black"> First I shall explore and clean the data files provided for the project. Following table contains the 11 files categorized by the data sources. </p> <br>

|      __IMDB__                |    __Box Office Mojo__   |   __Rotten Tomatoes__  | __TheMovieDB.org__  |
|------------------------------|--------------------------|------------------------|---------------------|
| 1. imdb.name.basics.csv      | 7. bom.movie_gross.csv   | 8. rt.movie_info.tsv   | 10. tmdb.movies.csv |
| 2. imdb.title.akas.csv       |                          | 9. rt.reviews.tsv      |                     |
| 3. imdb.title.basics.csv     |                          |                        |                     |
| 4. imdb.title.crew.csv       |                          |                        |                     |
| 5. imdb.title.principals.csv |                          |                        |                     |
| 6. imdb.title.ratings.csv    |                          |                        |                     |
|------------------------------|--------------------------|------------------------|---------------------|
|11. tn.movie_budgets.csv      |                          |                        |                     |

<p style="font-size:100%; color:green"> Methodology:  <b>R</b>OSEMED</p>
<!--><p style="color:black"> <b>R</b>OSEMED method </p> <-->

In [1]:
#importing libraries
import pandas as pd 
import numpy as np

# Data loading and cleaning

## cleaning the-numbers.com data file

In [2]:
# original source: https://www.the-numbers.com/movie/budgets
tn_budgets = pd.read_csv('Data/tn.movie_budgets.csv')
display(tn_budgets.head(4), len(tn_budgets))

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"


5782

In [3]:
# id column useless 
tn_budgets = tn_budgets.drop('id', axis = 1)
# format date
tn_budgets['release_date'] = pd.to_datetime(tn_budgets['release_date'])
# clean movie column
tn_budgets.movie = tn_budgets.movie.str.strip() 
# clean movie domestic_gross, production_budget and worldwide_gross
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace('$', '')
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace(',', '')
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].astype(float)
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace('$', '')
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace(',', '')
tn_budgets['production_budget'] = tn_budgets['production_budget'].astype(float)
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace('$', '')
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace(',', '')
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].astype(float)
# tn_budgets.isna().sum()  # checked it's clean

## cleaning Box Office Mojo data file

In [4]:
# Box Office Mojo dataset
bom = pd.read_csv('Data/bom.movie_gross.csv')
display(bom.head(2),len(bom))

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


3387

In [5]:
#bom.info()   # 3387 entries
# The Null values in domestic_gross and foreign_gross are tragged "-1.0 float64"
# and stidion with 'unknown'
bom['studio'].fillna(value='unknow',inplace=True)
bom['domestic_gross'] = pd.to_numeric(bom['domestic_gross'],errors='coerce')
bom['domestic_gross'].fillna(value=-1.0,inplace=True)
bom['foreign_gross'] = pd.to_numeric(bom['foreign_gross'],errors='coerce')
bom['foreign_gross'].fillna(value=str(-1.0),inplace=True)
bom = bom[bom.foreign_gross != -1.0]
#bom.isna().sum() # it's clean now

## cleaning Rotten Tomatoes data files

In [6]:
# Rotten Tomatoes
rt = pd.read_csv('Data/rt.movie_info.tsv', sep='\t')
display(rt.head(2),len(rt))

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One


1560

In [7]:
# we'll drop ['synopsis', 'genre', 'box_office']
rt.drop(columns=['synopsis', 'genre', 'box_office', 
                'director', 'writer', 'dvd_date', 'currency', 'box_office', 'studio'], inplace=True) # has much more 
# comprihencive data in IMDB
rt['rating'] = pd.to_numeric(rt['rating'],errors='coerce')
rt['rating'].fillna(value=-1.0,inplace=True)
rt = rt[rt.rating != -1.0]
rt['runtime']=rt.runtime.map(lambda x: x.strip().replace("minutes",""), na_action='ignore')
rt['runtime'] = pd.to_numeric(rt['runtime'],errors='coerce')
rt['runtime'].fillna(value=-1.0,inplace=True)
rt['theater_date'].fillna(value='unknown',inplace=True)
#rt.isna().sum() # It's clean now

Series([], Name: theater_date, dtype: object)

In [8]:
# review's file maybe useful for reviews and ratings. 
# encoding worked!!! with correct delimiter
rt_reviews = pd.read_csv('Data/rt.reviews.tsv', delimiter='\t', encoding='latin-1')
display(rt_reviews.head(2), len(rt_reviews))

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"


54432

In [9]:
#print(rt_reviews.isna().sum())
#display("length -->", len(rt_reviews))
#rt_reviews.dropna() # this doesn't work at all. 
#print(rt_reviews.isna().sum())
#display("length -->", len(rt_reviews))
# too many Nulls. No point using this DataSet

## cleaning IMDB data files

In [10]:
# IMDB informantion about movie personel 
name = pd.read_csv('Data/imdb.name.basics.csv')
display(name.head(3), len(name))

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"


606648

In [11]:
#print(name.isna().sum()) # birth_year(523912), death_year(599865) Null so dropping both,
#primary_profession(51340), known_for_titles(30204) will keep because maybe importat to 
# always can scrape for it later 
name = name.drop(['birth_year', 'death_year'], axis = 1)

In [12]:
# IMDB informantion about titles 
takas = pd.read_csv('Data/imdb.title.akas.csv')
takas.rename(columns = {'title_id':'imdb_id'}, inplace = True)
display(takas.head(3), len(takas))

,imdb_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0


331703

In [13]:
# checking data reducdencies
rand_id=np.random.randint(0,len(takas))
title=takas.loc[rand_id,'imdb_id']
display(takas.loc[(takas.imdb_id == title)])
'''checked with several random imdb_id's but all had totally convoluted data. the titles for the 
same imdb_id is completely different in most cases. This imdb.title.akas.csv data file is 
unsalvageable. file should not be used for further analysis.'''

,imdb_id,ordering,title,region,language,types,attributes,is_original_title
269923,tt7403456,1,Sharks and the City: New York,US,NaN,NaN,NaN,0.0


"checked with several random imdb_id's but all had totally convoluted data. the titles for the \nsame imdb_id is completely different in most cases. This imdb.title.akas.csv data file is \nunsalvageable. file should not be used for further analysis."

In [14]:
# IMDB datafile of title information could be important 
tbasics = pd.read_csv('Data/imdb.title.basics.csv')
tbasics.rename(columns = {'tconst':'imdb_id'}, inplace = True)
tbasics.rename(columns = {'start_year':'year'}, inplace = True)
display(tbasics.head(2),len(tbasics))

,imdb_id,primary_title,original_title,year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


146144

In [15]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(tbasics))   # this is a really good way to randamly sample the file 
title=tbasics.loc[rand_id,'imdb_id']
tbasics.loc[(tbasics.imdb_id == title) ]
# checked many imdb_id's and data is good and consistant. need some cleaning 

,imdb_id,primary_title,original_title,year,runtime_minutes,genres
44991,tt2667914,Pepe el andaluz,Pepe el andaluz,2013,84.0,"Documentary,Drama,Family"


In [16]:
# clean runtime_minutes(31739), genres(5408). I wouldn't drop any null lines 
# as primary title is all non-null
# no cleaning needed
tbasics['primary_title']=tbasics.primary_title.map(lambda x: x.strip(), na_action='ignore')
tbasics['original_title']=tbasics.original_title.map(lambda x: x.strip(), na_action='ignore')
tbasics['original_title'].fillna(value='unknown',inplace=True)
tbasics['year'] = pd.to_numeric(tbasics['year'],errors='coerce')
tbasics['year'].fillna(value=3001 ,inplace=True)
tbasics['runtime_minutes'] = pd.to_numeric(tbasics['runtime_minutes'],errors='coerce')
tbasics['runtime_minutes'].fillna(value=-1.0 ,inplace=True)
tbasics['genres']=tbasics.genres.map(lambda x: x.strip(), na_action='ignore')
tbasics['genres'].fillna(value='unknown',inplace=True)
#tbasics.isna().sum() # it's clean
# save the cleaned file 
#tbasics.to_csv('Data/imdb.title.basics_clean.csv', index=False)

In [17]:
# IMDB all crew information per title needs imdb.name.basics.csv file 
crew = pd.read_csv('Data/imdb.title.crew.csv')
crew.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(crew.head(2),len(crew))

,imdb_id,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


146144

In [18]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(crew))   
title=crew.loc[rand_id,'imdb_id']
crew.loc[(crew.imdb_id == title) ]

,imdb_id,directors,writers
112352,tt3373262,nm3809488,"nm3027289,nm3905570,nm3809488"


In [19]:
#print("before\n", crew.isna().sum())
crew['directors']=crew.directors.map(lambda x: x.strip(), na_action='ignore')
crew['writers']=crew.writers.map(lambda x: x.strip(), na_action='ignore')
crew['writers'].fillna(value='unknown',inplace=True)
crew = crew[crew.writers != 'unknown']
crew['directors'].fillna(value='unknown',inplace=True)
crew = crew[crew.directors != 'unknown']
#print("after\n", crew.isna().sum())
# save the cleaned file 
#tbasics.to_csv('Data/imdb.title.crew_clean.csv', index=False)

In [20]:
# IMDB ratings 
ratings = pd.read_csv('Data/imdb.title.ratings.csv')
ratings.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(ratings.head(2), len(ratings))

,imdb_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


73856

In [21]:
# check if the imdb_id is consistant here
rand_id=np.random.randint(0,len(ratings))   
title=ratings.loc[rand_id,'imdb_id']
ratings.loc[(ratings.imdb_id == title) ]

,imdb_id,averagerating,numvotes
65252,tt2117979,3.9,117


In [22]:
#print("before:\n", ratings.isna().sum())
ratings['averagerating'] = pd.to_numeric(ratings['averagerating'],errors='coerce')
ratings['numvotes'] = pd.to_numeric(ratings['numvotes'],errors='coerce')
#print("after:\n", ratings.isna().sum()) # looks pretty clean
ratings.to_csv('Data/imdb.title.ratings_clean.csv', index=False)

In [23]:
# this data maybe redundent 
principals = pd.read_csv('Data/imdb.title.principals.csv')
principals.rename(columns = {'tconst':'imdb_id'}, inplace = True)
display(principals.head(2),len(principals))

,imdb_id,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN


1028186

In [24]:
#print("before:\n", principals.isna().sum())
principals.dropna(inplace=True) # null after cleaning. data is useless 
#print("after:\n", principals.isna().sum()) # looks pretty clean
#display(principals.head(2),len(principals))

## TheMovieDB data file

In [25]:
# TheMovieDB.org
tmdb = pd.read_csv('Data/tmdb.movies.csv')

# we'll drop ['Unnamed: 0', 'genre_ids', 'popularity']
tmdb.drop(columns=['Unnamed: 0', 'genre_ids', 'popularity'], inplace=True) 
display(tmdb.head(3), len(tmdb))

,id,original_language,original_title,release_date,title,vote_average,vote_count
0,12444,en,Harry Potter and the Deathly Hallows: Part 1,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,10191,en,How to Train Your Dragon,2010-03-26,How to Train Your Dragon,7.7,7610
2,10138,en,Iron Man 2,2010-05-07,Iron Man 2,6.8,12368


26517

In [26]:
# comprihencive data in TMDB
tmdb['original_language']=tmdb.original_language.map(lambda x: x.strip() , na_action='ignore')
tmdb['original_title']=tmdb.original_title.map(lambda x: x.strip() , na_action='ignore')
tmdb['release_date']=tmdb.release_date.map(lambda x: x.strip() , na_action='ignore')
tmdb['title']=tmdb.title.map(lambda x: x.strip() , na_action='ignore')

#print("before:\n", tmdb.isna().sum())
tmdb.dropna(inplace=True) # null after cleaning. data is useless 
#print("after:\n", tmdb.isna().sum()) # looks pretty clean now

# Alternative data from IMDBpro
credit: Jesse Newman for web scraping

In [27]:
region=pd.read_csv('Data/region.csv')
revenue=pd.read_csv('Data/revenue.csv')
genres=pd.read_csv('Data/genres.csv')
date=pd.read_csv('Data/date.csv')
# credit: Jesse Newman for web scraping

In [28]:
#display(region.head(2))
#display(revenue.head(2))
#display(genres.head(2))
#display(date.head(2))

In [29]:
display(region.loc[region.imdb_id.isnull()==True])
display(revenue.imdb_id.isnull().sum())
display(genres.imdb_id.isnull().sum())
display(date.imdb_id.isnull().sum())

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross
7850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0

0

0

In [30]:
region = region.drop(7850)

In [31]:
# removing redundant cols 
region.drop(['director', 'production_co', 'rank', 'budget_usd', 'us_gross'], axis = 1, inplace=True)
#region = region.imdb_id.duplicated(keep='first')
display(region.head(2))

revenue.drop('rank', axis = 1, inplace=True)

,imdb_id,title,year,region_code
0,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,[US]
1,tt4154796,Avengers: Endgame,2019,[US]


In [32]:
# clean region_code
region['region_code'] = region['region_code'].str.replace('\[', '')
region['region_code'] = region['region_code'].str.replace('\]', '')
display(region.head(2))

,imdb_id,title,year,region_code
0,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,US
1,tt4154796,Avengers: Endgame,2019,US


In [33]:
region.loc[region.region_code.isnull() == True]

,imdb_id,title,year,region_code
320,tt0142342,Big Daddy,1999,NaN
432,tt0119217,Good Will Hunting,1997,NaN
441,tt0120685,Godzilla,1998,NaN
446,tt0100758,Teenage Mutant Ninja Turtles,1990,NaN
491,tt0190332,"Crouching Tiger, Hidden Dragon",2000,NaN
...,...,...,...,...
14677,tt0094671,Apprentice to Murder,1988,NaN
14680,tt0099053,Archangel,1990,NaN
14681,tt0129758,Arguing the World,1997,NaN
14682,tt6097926,Arrangiarsi: Pizza... & the Art of Living,2017,NaN


In [34]:
region.region_code.fillna(value = 'unknown', inplace = True) # some of the movies are obviously made in US. Thus I labelled it 
# as 'unknown' so that they maybe included in the analysis once other method can be used to verify their origin

In [35]:
# clean revenue year column 
cnt = 0
for index in range(len(revenue)):
    try:
        clean_year=revenue.iloc[index]['year']
        int(clean_year)
    except ValueError:
        #print(index, revenue.iloc[index]['year'])
        # clean it.
        #clean_year=clean_year.split("-",1)
        revenue.at[index, 'year'] = str(clean_year[0:4])
        #revenue.iloc[index]['year'] = str(clean_year[0:4])
        #print("new year =",clean_year[0:4])
    cnt+=1

revenue['year'] = revenue['year'].astype('int64')
revenue['us_gross'] = revenue['us_gross'].astype('float64')

In [36]:
#display(region.info())
#display(revenue.info())
#display(genres.info())
#display(date.info())

## is region important ? 
Since Microsoft is a US based company it may be predent to select US based movies. Let's inspect the data more closely.

In [37]:
region.loc[region.region_code != 'US'].head(10)

,imdb_id,title,year,region_code
35,tt2283362,Jumanji: Welcome to the Jungle,2017,PH
109,tt3470600,Sing,2016,JP
149,tt3183660,Fantastic Beasts and Where to Find Them,2016,GB
181,tt1727824,Bohemian Rhapsody,2018,GB
216,tt2379713,Spectre,2015,GB
232,tt5013056,Dunkirk,2017,GB
280,tt0090555,Crocodile Dundee,1986,AU
320,tt0142342,Big Daddy,1999,unknown
331,tt0246460,Die Another Day,2002,GB
335,tt4123430,Fantastic Beasts: The Crimes of Grindelwald,2018,GB


The 1^st random example <b>"Jumanji: Welcome to the Jungle"</b>  designated as PH in this data is clearly wrong. It's a US based movie and became quite popular critically as well as commercially. Also <b>"spectre"</b>, the 007 series moview is a hugely popular movie. It seems from the first glasnce it may be mileading to filter out movies that are not designated as US.

## Movie budget and gross revenue 

In [38]:
# budget_usd   us_gross
#display(revenue.loc[revenue.budget_usd == -1]['budget_usd'].count())
display(revenue.loc[revenue.budget_usd == -1])
display(revenue.budget_usd.describe())
display(revenue.loc[revenue.us_gross < 0]['us_gross'].count())
display(revenue.us_gross.describe())

,imdb_id,title,year,director,production_co,budget_usd,us_gross
304,tt0486946,Wild Hogs,2007,Walt Becker,Touchstone Pictures,-1.0,168000000.0
559,tt0082846,On Golden Pond,1981,Mark Rydell,IPC Films,-1.0,119000000.0
595,tt0091326,The Karate Kid Part II,1986,John G. Avildsen,Columbia Pictures,-1.0,115000000.0
712,tt0034492,Bambi,1942,James Algar,Walt Disney Animation Studios,-1.0,102000000.0
726,tt0081562,Stir Crazy,1980,Sidney Poitier,Columbia Pictures,-1.0,101000000.0
...,...,...,...,...,...,...,...
14421,tt7615722,The Legend of Hallowaiian,2018,Sean Patrick O'Reilly,Arcana Studio,-1.0,451.0
14423,tt1757742,Apartment 143,2011,Carles Torrens,Nostromo Pictures,-1.0,383.0
14425,tt0462302,Intervention,2007,Mary McGuckian,Scion Films,-1.0,279.0
14427,tt4796122,Satanic,2016,Jeffrey G. Hunt,Magnet Releasing,-1.0,252.0


count    1.443100e+04
mean     4.908767e+08
std      1.154901e+10
min     -1.000000e+00
25%     -1.000000e+00
50%      1.000000e+06
75%      1.800000e+07
max      8.198500e+11
Name: budget_usd, dtype: float64

0

count    1.443100e+04
mean     2.052120e+07
std      4.948649e+07
min      3.000000e+01
25%      1.010000e+05
50%      1.500000e+06
75%      1.900000e+07
max      9.370000e+08
Name: us_gross, dtype: float64

To understand the profitability of the movie both gross revenue and budget is needed. In this dataset with 14,431 entries, 6,534 entries do not have budget information. However, movies like <b>Wild Hogs</b> (\$ 168 million) and <b>Bambi</b> (\$ 102 million) clearly had boxoffice success. Thus, eliminating those data at this stage might not be predent. 

## Join region, genre, date, runtime and rating to  revenue DataFrame

In [39]:
# joining region to the revenue table 
region.drop(['title', 'year'], axis = 1, inplace=True)
region.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)

# join gave a weird bug. joined table didn't have the correct number of records
# so following code joining the tables brute force correctly
j=0
Row_list =[] 
mylist = []
for i in range(len(revenue)):
    #i=j
    while  revenue.iloc[i]['imdb_id'] != region.iloc[j]['imdb_id']:
        j+=1
    if revenue.iloc[i]['imdb_id'] == region.iloc[j]['imdb_id']:
        imdbid=region.iloc[j]['imdb_id']
        code=region.iloc[j]['region_code']
        mylist=[imdbid, code]
        Row_list.append(mylist) 

df = pd.DataFrame(Row_list, columns= region.columns)

# now the join should exactly teild 14,431 records 
df.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(df, how='right', lsuffix='1', rsuffix='1',  sort=True)
df.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)

In [40]:
# joining genres to the revenue table 
genres.drop(['title', 'year'], axis = 1, inplace=True)
genres.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
genres.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(genres, how='left', lsuffix='1', rsuffix='1',  sort=True)
genres.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
#display(revenue.head(3))

In [41]:
# joining date to the revenue table 
date.drop(['title'], axis = 1, inplace=True)
date.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
date.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(date, how='left', lsuffix='1', rsuffix='1',  sort=True)
date.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
# clean date
revenue['date'] = revenue['date'].astype('datetime64[ns]')
#display(revenue.head(3))

In [42]:
# joining runtime to the revenue table 
tbasics.drop(['primary_title', 'original_title', 'year', 'genres'], axis = 1, inplace=True)
tbasics.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
tbasics.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(tbasics, how='left', lsuffix='1', rsuffix='1',  sort=True)
tbasics.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
#display(revenue.head(3))

In [43]:
# joining ratings to the revenue table 
ratings.drop(['numvotes'], axis = 1, inplace=True)
ratings.sort_values(by=['imdb_id'], inplace=False)
revenue.sort_values(by=['imdb_id'], inplace=False)
ratings.set_index('imdb_id', inplace=True)
revenue.set_index('imdb_id', inplace=True)
revenue = revenue.join(ratings, how='left', lsuffix='1', rsuffix='1',  sort=True)
ratings.reset_index('imdb_id', inplace=True)
revenue.reset_index('imdb_id', inplace=True)
#display(revenue.head(3))

In [44]:
# above 3 raws lack both budget and genre information. This dropping
revenue=revenue[revenue.genres != '\\N']
display(revenue.head(2), len(revenue))

,imdb_id,title,year,director,production_co,budget_usd,us_gross,region_code,genres,date,runtime_minutes,averagerating
0,tt0012190,The Four Horsemen of the Apocalypse,1921,Rex Ingram,Metro Pictures Corporation,800000.0,9200000.0,US,"Drama,Romance,War",1921-03-06,NaN,NaN
1,tt0012494,Destiny,1921,Fritz Lang,Decla-Bioscop AG,-1.0,12000.0,DE,"Drama,Fantasy,Thriller",1921-10-06,NaN,NaN


14428

In [45]:
revenue.to_csv('Data/movie_main.csv', index=False)

In [46]:
revenue.head(10)

,imdb_id,title,year,director,production_co,budget_usd,us_gross,region_code,genres,date,runtime_minutes,averagerating
0,tt0012190,The Four Horsemen of the Apocalypse,1921,Rex Ingram,Metro Pictures Corporation,800000.0,9200000.0,US,"Drama,Romance,War",1921-03-06,NaN,NaN
1,tt0012494,Destiny,1921,Fritz Lang,Decla-Bioscop AG,-1.0,12000.0,DE,"Drama,Fantasy,Thriller",1921-10-06,NaN,NaN
2,tt0015648,Battleship Potemkin,1925,Sergei M. Eisenstein,Goskino,-1.0,51000.0,SUHH,"Drama,History,Thriller",1925-12-24,NaN,NaN
3,tt0017136,Metropolis,1927,Fritz Lang,Universum Film (UFA),0.0,1200000.0,DE,"Drama,Sci-Fi",1927-01-10,NaN,NaN
4,tt0018737,Pandora's Box,1929,Georg Wilhelm Pabst,Nero-Film AG,-1.0,53000.0,DE,"Crime,Drama,Romance",1929-01-30,NaN,NaN
5,tt0019254,The Passion of Joan of Arc,1928,Carl Theodor Dreyer,Société générale des films,-1.0,22000.0,FR,"Biography,Drama,History",1928-04-21,NaN,NaN
6,tt0021577,L'Age d'Or,1930,Luis Buñuel,Vicomte de Noailles,-1.0,33000.0,FR,"Comedy,Drama",1930-10-28,NaN,NaN
7,tt0021749,City Lights,1931,Charles Chaplin,Charles Chaplin Productions,1500000.0,19000.0,US,"Comedy,Drama,Romance",1931-02-01,NaN,NaN
8,tt0022100,M,1931,Fritz Lang,Nero-Film AG,-1.0,36000.0,DE,"Crime,Mystery,Thriller",1931-05-11,NaN,NaN
9,tt0022877,Fanny,1932,Marc Allégret,Les Films Marcel Pagnol,-1.0,8000.0,FR,"Comedy,Drama",1932-11-02,NaN,NaN
